In [5]:
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools import cache
from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, sortdict, data, quantify, atom, atoms, list_atoms, list_multiply, mapt, Node
from aocutils.grid import TURNRIGHT, TURNLEFT, move, gridneigh, arr_to_dict, grid_to_dict, neighbors, complexneighbors, arr_neighbors, iterate, Dim, dimensions, positive, ranges2grid, manhattan, conv1d, conv2d, rotate, rot90
from itertools import product
import networkx as nx
E = (0,1); W = (0,-1); N = (-1,0); S = (1,0)
winds = [N, E, S, W]
inf = float('inf')

In [6]:
   
@cache
def direcnext(start, goal):
    # goes to a button and back to A
    
    keys = {'^':(0,1), 'A':(0,2), '<':(1,0), 'v':(1,1), '>':(1,2)}
    
    # print('pressing from', loc, 'goal', keys[char])
    G = nx.Graph()
    for loc1 in keys.values():
        for loc2 in keys.values():
            if abs(loc1[0]-loc2[0]) + abs(loc1[1]-loc2[1]) == 1:
                G.add_edge(loc1, loc2)
    options = []
    return tuple(nx.all_shortest_paths(G, keys[start], keys[goal]))
        
        # for waytoA in nx.all_shortest_paths(G, keys[goal], keys['A']):
            # options.append(path + ['A'] + waytoA[1:])
    return options

@cache 
def numericnext(start, goal):
    keys = {'7':(0,0), '8':(0,1), '9':(0,2), '4':(1,0), '5':(1,1), '6': (1,2), '1':(2,0), '2':(2,1), '3':(2,2), '0':(3,1), 'A':(3,2)}
    # gives back options to next step
    G = nx.Graph()
    for loc1 in keys.values():
        for loc2 in keys.values():
            if abs(loc1[0]-loc2[0]) + abs(loc1[1]-loc2[1]) == 1:
                G.add_edge(loc1, loc2)
    return tuple(nx.all_shortest_paths(G, keys[start], keys[goal]))

def numerictotal(goalstring):
    # given a string, comes up with paths how to type in that string
    paths = [[]]
    for a,b in zippify('A'+goalstring, 2):
        newpaths = []
        for p1 in paths:
            for p2 in numericnext(a,b):
                newpaths.append(p1.copy() + p2.copy()[1:] + ['press'])
        paths = newpaths
    return tuple(paths)
def directtotal(goalstring):
    # given instructions for a directional robot, comes up with paths how to type in that string
    paths = [[]]
    
    for a,b in zippify(['A']+goalstring, 2):
        newpaths = []
        if a == b:
            for p1 in paths:
                newpaths.append(p1.copy() + ['press'])
        else:
            for p2 in direcnext(a,b):
                # print('from to', a,b,'path', p2)
                for p1 in paths:
                    newpaths.append(p1.copy() + p2.copy()[1:] + ['press'])
        paths = newpaths
    return paths
            
        
def numericpath_to_direcpath(path):
    # this essentially gives the goal for the next robot
    needed = []
    a = (3,2)
    for b in path:
        if b == 'press': needed.append('A')
        else:
            assert abs(a[0] - b[0]) + abs(a[1] - b[1]) == 1
            if a[0] - b[0] == 1: needed.append('^')
            elif a[0] - b[0] == -1:needed.append('v')
            elif a[1] - b[1] == 1:needed.append('<')
            elif a[1] - b[1] == -1:needed.append('>')
            else: print('error')
            a = b
    return (needed)

def direcpath_to_directpath(path):
    # this essentially gives the goal for the next robot
    needed = []
    a = (0,2)
    for b in path:
        if b == 'press': needed.append('A')
        else:
            assert abs(a[0] - b[0]) + abs(a[1] - b[1]) == 1
            if a[0] - b[0] == 1: needed.append('^')
            elif a[0] - b[0] == -1:needed.append('v')
            elif a[1] - b[1] == 1:needed.append('<')
            elif a[1] - b[1] == -1:needed.append('>')
            else: print('error')
            a = b
    return (needed)
 


In [13]:
# these are the keypresses robot 1 needs to do
# works on example but not on 25 robots
f = open("in.txt").read().split("\n")
lines = [line for line in f]
lines[0]

@cache
def explode(smallpart):
    i = smallpart
    smallpart = flatten([mapping[a] for a in zippify((smallpart))])
    for step in range(14):
        smallpart = flatten([mapping[a] for a in zippify(['A']+list(smallpart))])
    
    return len(smallpart)

ans = 0
for line in lines:
    minn = inf
    for p0 in numerictotal(line):
        ins = numericpath_to_direcpath(p0)
        for step in range(10):
            ins = flatten([mapping[a] for a in zippify(['A']+ins,2)])
        # minn = min(len(ins), minn)

        temp = 0
        for smallpart in zippify(['A']+ins,2):
            temp += explode(smallpart)
        minn = min(minn, temp)
            
    print('found', minn, line[:-1])
    ans += minn * int(line[:-1])
ans
    

found 87288844796 463
found 84248089342 340
found 90594397580 129
found 80732180764 083
found 90750571880 341


118392478819140

In [711]:
# this code is to fill the mapping that is used later on. Unfortunately it's wrong since not all the paths are optimal
f = open("t0.txt").read().split("\n")
lines = [line for line in f]
lines[0]

mapping = {}
ans = 0
for line in lines:
    minn = inf
    
    for p0 in numerictotal(line):
        
        ins = numericpath_to_direcpath(p0)
        print(ins)
        
        options = directtotal(ins)
        best = min(len(o) for o in options)
        for p1 in options:
            if len(p1) == best:
                ins2 = direcpath_to_directpath(p1)
                parse(ins, ins2)
                o2 = directtotal(ins2)
                besto2 = min(len(o) for o in o2)
                for p2 in o2:
                    if len(p2) == besto2:
                        p3 = direcpath_to_directpath(p2)
                        # parse(ins2,p3)
                        minn = min(len(p2), minn)
    print(minn, line[:-1])
    ans += minn * int(line[:-1])
ans
    

['<', 'A', '^', 'A', '^', '^', '>', 'A', 'v', 'v', 'v', 'A']
value already in from A to < ['v', '<', '<', 'A'] != ['<', 'v', '<', 'A']
value already in from A to < ['<', 'v', '<', 'A'] != ['v', '<', '<', 'A']
value already in from < to A ['>', '>', '^', 'A'] != ['>', '^', '>', 'A']
value already in from A to < ['v', '<', '<', 'A'] != ['<', 'v', '<', 'A']
value already in from A to < ['<', 'v', '<', 'A'] != ['v', '<', '<', 'A']
value already in from < to A ['>', '^', '>', 'A'] != ['>', '>', '^', 'A']
value already in from ^ to > ['v', '>', 'A'] != ['>', 'v', 'A']
value already in from A to < ['v', '<', '<', 'A'] != ['<', 'v', '<', 'A']
value already in from A to < ['<', 'v', '<', 'A'] != ['v', '<', '<', 'A']
value already in from < to A ['>', '>', '^', 'A'] != ['>', '^', '>', 'A']
value already in from A to < ['v', '<', '<', 'A'] != ['<', 'v', '<', 'A']
value already in from A to < ['<', 'v', '<', 'A'] != ['v', '<', '<', 'A']
value already in from < to A ['>', '^', '>', 'A'] != ['>', '>

126384